# Проект: pretrain и posttrain LLM

## Постановка задачи

- Pretrain
    - Претрейн — самый ресурсоёмкий этап обучения LLM. Чтобы полноценно обучить даже небольшую модель (менее 1B), понадобится более 10к GPU-часов на A100. Чтобы не тратить недели на обучение, но отработать ключевые приёмы, в проекте вы выполните упрощённую задачу. 
    - При полноценном претрейне модель учится обобщать знания из данных, на которых происходило обучение, чтобы потом извлекать эти знания по текстовым запросам уже после обучения. Упростим задачу — научим модель только структуре языка. 
    - Сосредоточимся на одном узком домене — **текстах произведений русской литературы** — и **обучим модель продолжать фразы из этого домена разумным текстом**. 
- Posttrain
    - Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель **Qwen2.5-0.5B**, с которой вы встречались в уроках. **Обучите её генерировать ответы на инструктивные русскоязычные вопросы**.

## Критерии успеха
- Pretrain - чтобы оценить качество, используйте промпты:
    - "Все мысли, которые имеют огромные последствия",
    - "Сила войска зависит от его духа",
    - "Мысль о том, что он принес страдания",
    - "Человек сознает себя свободным",
    - "Что бы ни случилось, я всегда буду",
    - "Любовь мешает смерти",
    - "Нет, жизнь не кончена",
    - "Всякая мысль, даже самая простая",
    - "Война не любезность, а самое гадкое дело",
    - "Чтобы жить честно"
- Posttrain - качество данных должно быть сопоставимо с таким вариантом::
    - Model Input 1:
    сколько планет в нашей солнечной системе?
        - Model Output 1:
    Согласно последним исследованиям, Солнце имеет 8 планеты. Это дает планетам, которые соответствуют разным критериям, таким как диета, местоположение, плотность, климат и другие.
    assistant
    Общее размер нашей Солнечной системы составляет примерно 9,9 миллиарда километров, а в результате на каждом из них есть планета,
    - Model Input 2:
    расскажи стих
        - Model Output 2:
    Вот сладкий сладкий вкус, который ты получаешь на себе
    Когда я говорю, что я люблю тебя
    Мое сердце, у меня есть все, что я хочу, чтобы сделать
    У меня есть все, что мне нужно, чтобы быть хорошим
    Так давай пойдем в ресторан
    assistant
    Пока мы уседим, что мы можем
    И в этот момент мы возвращаемся к при
    - Model Input 3:
    когда собирать крыжовник?
        - Model Output 3:
    Когда собираешь крыжовник, важно не беспокоиться о том, что другие могут его хвастаться. Вместо этого проверяйте свои способности и готовность. Вы можете изучить свой талант и стремления, прежде чем принимать решение, и признавать, что у вас есть свои сильные стороны, а также слабые места. Не снимайте крыжовник в обществе, и обязательно помните о
    - Model Input 4:
    Как быстро выучить новый язык?
        - Model Output 4:
    Скорость усвоения нового языка может сильно различаться в зависимости от нескольких факторов, таких как скорость, с которой вы пакетируете информацию через ее и ее способность учиться в различных условиях. Тем не менее, вот некоторые общие рекомендации: 1. Делайте сценарии. Обычно это может занять от 2 до 4 лет, чтобы понять основные концепции. Вы можете начать с

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных

- Pretrain
    - https://github.com/JoannaBy/RussianNovels/tree/master/corpus 
- Posttrain
    - русскоязычный инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) в диалоговом формате

## Инициализация

### Импорт библиотек

### Установка главных параметров

In [1]:


"""
ENCODER_NAME = "cointegrated/rubert-tiny2"
#ENCODER_NAME = "sentence-transformers/all-MiniLM-L6-v2"
TOKENIZER_MAX_LEN = 512
RELATIONS_MAX_LEN = 30 # TOP отношений
EPOCHS_MAX = 3000
LEARNING_RATE = 5e-5
##v2 LEARNING_RATE = 5e-4 - укрупнение шага не помогло
TEXT_LR = 1e-5
##v2 TEXT_LR = 1e-4 - укрупнение шага не помогло
DROPOUT = 0.1
WEIGHT_DECAY = 1e-4
SCHEDULER_T_MAX = 10
SCHEDULER_ETA_MIN = 1e-6
BATCH_SIZE = 32
TARGET_TOKEN_MACRO_F1 = 0.5
TARGET_CLS_MICRO_F1 = 0.8
"""

'\nENCODER_NAME = "cointegrated/rubert-tiny2"\n#ENCODER_NAME = "sentence-transformers/all-MiniLM-L6-v2"\nTOKENIZER_MAX_LEN = 512\nRELATIONS_MAX_LEN = 30 # TOP отношений\nEPOCHS_MAX = 3000\nLEARNING_RATE = 5e-5\n##v2 LEARNING_RATE = 5e-4 - укрупнение шага не помогло\nTEXT_LR = 1e-5\n##v2 TEXT_LR = 1e-4 - укрупнение шага не помогло\nDROPOUT = 0.1\nWEIGHT_DECAY = 1e-4\nSCHEDULER_T_MAX = 10\nSCHEDULER_ETA_MIN = 1e-6\nBATCH_SIZE = 32\nTARGET_TOKEN_MACRO_F1 = 0.5\nTARGET_CLS_MICRO_F1 = 0.8\n'